In [1]:
import pandas as pd
from matplotlib import pyplot as plt 
import numpy as np
from surprise import Dataset
from surprise import Reader
from surprise import KNNWithMeans

## Constructing base model

In [2]:
# choosing only the columns that are needed. The column 'Review' contains the text that makes the dataframe heavy. So it is dropped.
reviews = pd.read_csv('../data/reviews.csv', usecols=['AuthorId', 'ReviewId', 'RecipeId', 'Rating'])
# Data analysis shows 'Rating'=0 is not neccesarily bad rating 
# (Refer to https://www.kaggle.com/code/gemmin/sentiment-analysis)
# Removing 0 ratings
reviews = reviews[reviews['Rating']!=0]
# Only reviews by active reviewers (selected_authors) are accepted, as the model will be based on active reviewers.
# Active reviewer is whoever has more than a minimum number of reviews in the data
review_counts = reviews.groupby('AuthorId').count()
selected_authors = review_counts.loc[review_counts['ReviewId']>=10].index.values
reviews = reviews.loc[reviews['AuthorId'].isin(selected_authors)]

print('number of selected reviews:', reviews['ReviewId'].nunique())
print('number of selected reviewers:', reviews['AuthorId'].nunique())
print('number of selected recipes:', reviews['RecipeId'].nunique())

number of selected reviews: 978879
number of selected reviewers: 14544
number of selected recipes: 238699


In [9]:
reader = Reader(rating_scale=(1, 5))

# Loads Pandas dataframe
data = Dataset.load_from_df(reviews.loc[:100000, ["AuthorId", "RecipeId", "Rating"]], reader)
# To use user-based cosine similarity
sim_options = {
    "name": "cosine",
    "user_based": True,  # Compute  similarities between users
}
algo = KNNWithMeans(sim_options=sim_options)

In [10]:
trainingSet = data.build_full_trainset()
algo.fit(trainingSet)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [11]:
prediction = algo.predict(1634, 5466)
prediction.est

4.970443349753695

## Parameter Tuning

In [ ]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise.model_selection import GridSearchCV

data = Dataset.load_builtin("ml-100k")
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])